In [12]:
# -*- coding: utf-8 -*-
"""
Created on Wed Dec 18 09:15:05 2019

@author: Frank
"""

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession

from pyspark.sql.functions import regexp_extract


In [13]:
# SparkSession을 생성합니다. (config 부분은 Windows에서만 필요합니다!)
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "file:///C:/temp").appName("StructuredStreaming").getOrCreate()

In [14]:
# logs 디렉토리를 모니터링하여 새로운 로그 데이터를 읽어들여 accessLines 변수에 저장합니다.
accessLines = spark.readStream.text("logs")
accessLines

Py4JJavaError: An error occurred while calling o53.text.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:180)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anonfun$map$1(Trave

In [ ]:
# Parse out the common log format to a DataFrame
# 공통 로그 형식을 DataFrame으로 파싱합니다.
contentSizeExp = r'\s(\d+)$' # 정규 표현식을 사용하여 content size를 추출합니다.
statusExp = r'\s(\d{3})\s' # 정규 표현식을 사용하여 상태 코드를 추출합니다.
generalExp = r'\"(\S+)\s(\S+)\s*(\S*)\"' # 정규 표현식을 사용하여 메소드, 엔드포인트, 프로토콜을 추출합니다.
timeExp = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]' # 정규 표현식을 사용하여 타임스탬프를 추출합니다.
hostExp = r'(^\S+\.[\S+\.]+\S+)\s' # 정규 표현식을 사용하여 호스트를 추출합니다.

logsDF = accessLines.select(regexp_extract('value', hostExp, 1).alias('host'), # value에서 호스트를 추출하여 'host' 열로 지정합니다.
                         regexp_extract('value', timeExp, 1).alias('timestamp'), # value에서 타임스탬프를 추출하여 'timestamp' 열로 지정합니다.
                         regexp_extract('value', generalExp, 1).alias('method'), # value에서 메소드를 추출하여 'method' 열로 지정합니다.
                         regexp_extract('value', generalExp, 2).alias('endpoint'), # value에서 엔드포인트를 추출하여 'endpoint' 열로 지정합니다.
                         regexp_extract('value', generalExp, 3).alias('protocol'), # value에서 프로토콜을 추출하여 'protocol' 열로 지정합니다.
                         regexp_extract('value', statusExp, 1).cast('integer').alias('status'), # value에서 상태 코드를 추출하여 'status' 열로 지정합니다.
                         regexp_extract('value', contentSizeExp, 1).cast('integer').alias('content_size')) # value에서 content size를 추출하여 'content_size' 열로 지정합니다.


In [ ]:
# 각 상태 코드별로 액세스 횟수를 계산하기 위해 running count를 유지합니다.
statusCountsDF = logsDF.groupBy(logsDF.status).count()


In [ ]:
# 스트리밍 쿼리를 시작하고 결과를 콘솔에 출력합니다.
query = ( statusCountsDF.writeStream.outputMode("complete").format("console").queryName("counts").start() )


In [ ]:
# 영원히 실행되며 종료될 때까지 실행합니다.
query.awaitTermination()


In [ ]:
# Cleanly shut down the session
spark.stop()  # 세션을 정리하고 종료합니다.
